In [1]:
!pip install -U -q google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.8.0 which is incompatible.


# Mount drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Config

In [3]:
from google import genai
from google.genai import types
from PIL import Image
import os
import time

In [4]:
from google.colab import userdata

#Get API KEY from colab.
GOOGLE_API_KEY = userdata.get('GeminiAPI')
client = genai.Client(api_key=GOOGLE_API_KEY)

# Load Image

In [9]:
def load_image_from_subfolders(folder_path):
  image_files = []
  for root, _, files in os.walk(folder_path):  # os.walk iterates through subfolders
      for filename in files:
        if filename.endswith(('.jpg', '.jpeg', '.png')):
          image_files.append(os.path.join(root, filename))
  return image_files


In [10]:
# def load_image(folder_path):
#   image_files = []
#   for filename in os.listdir(folder_path):
#       if filename.endswith(('.jpg', '.jpeg', '.png')):
#           image_files.append(os.path.join(folder_path, filename))
#   return image_files


In [ ]:
folder_path = 'path'

In [ ]:
image_paths = load_image_from_subfolders(folder_path)
image_paths.sort()
image_paths

In [15]:
# image_paths = load_image(folder_path)
# image_paths.sort()
# image_paths

In [16]:
len(image_paths)

3915

In [17]:
import os

def convert_name(image_path):
  filename = os.path.basename(image_path)
  name, ext = os.path.splitext(filename)
  return name

In [18]:
# def convert_ID(path):
#   ID = int(path.split('/')[-1].split('.')[0]) #Use for EVJVQA
#   # ID = int(os.path.basename(path).split('_')[1].split('.')[0]) #Use for ViVQA4Edu
#   return ID

# Visual Question Answering

In [19]:
def extract_and_count(llms_output):
    # Tách chuỗi theo dòng
    lines = llms_output.split('\n')
    # Lọc bỏ các dòng không phải câu hỏi
    result = [line.strip() for line in lines if line.strip() and not line.strip().startswith('Ảnh:')]
    # Trả về danh sách câu hỏi và số lượng
    return result, len(result)


## Question Generation - Gemini

In [19]:
model_name = "gemini-2.0-flash"

ques_system_instructions = """
Chỉ phản hồi bằng Tiếng Việt và tuân thủ định dạng sau (Quan trọng! Chú ý phản hồi đúng định dạng):

Ảnh:
Câu hỏi 1: Nội dung câu hỏi 1,
Câu hỏi 2: Nội dung câu hỏi 2,
...,
Câu hỏi k: Nội dung câu hỏi k.
      """

safety_settings = [
    types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_ONLY_HIGH",
    ),
]

In [20]:
prompt_ques = """Bạn là một chuyên gia giàu kinh nghiệm trong lĩnh vực gán nhãn dữ liệu và tạo ra các câu hỏi để phân tích nội dung hình ảnh. Nhiệm vụ của bạn là đưa ra các câu hỏi về nội dung của bức ảnh dựa trên các yếu tố quan trọng như:
- Đối tượng chính trong ảnh (con người, động vật, đồ vật, phong cảnh).
- Hành động hoặc tình huống đang diễn ra.
- Ngữ cảnh xung quanh hoặc các yếu tố liên quan (màu sắc, vị trí, trạng thái, cảm xúc).

Hãy đảm bảo rằng các câu hỏi:
- Chỉ liên quan đến nội dung có trong bức ảnh.
- Có tính đa dạng về chủ đề (ví dụ: câu hỏi về đối tượng, hành động, hoặc bối cảnh).
- Nếu không thể đưa ra bất kỳ câu hỏi nào phù hợp, hãy phản hồi "Không thể tìm ra câu hỏi" cho ảnh tương ứng.

Hãy cẩn thận xem xét và đưa ra các câu hỏi có chất lượng cao nhất."""

In [21]:
imageID_list = []
question_list = []

start = 3500
end = 3915

for path in image_paths[start:end]:
  # ID = convert_ID(path)
  ID = convert_name(path)
  imageID_list.append(ID)
  print(ID) #Check
  image = Image.open(path)
  contents = [prompt_ques] + [image]

  try:
    response = client.models.generate_content(
        model = model_name,
        contents = contents,
        config = types.GenerateContentConfig(
            system_instruction=ques_system_instructions,
            temperature=0.4,
            safety_settings=safety_settings,
            )
        )

    try:
    # Check if 'candidates' list is not empty
      if response.candidates:
          # Access the first candidate's content if available
          if response.candidates[0].content.parts:
              generated_text = response.candidates[0].content.parts[0].text
              question_list.append(generated_text)
              time.sleep(3)
          else:
              print("No generated text found in the candidate.")
              question_list.append("Không có chú thích")
      else:
          print("No candidates found in the response.")
          question_list.append("Không nhận được phản hổi")
    except (AttributeError, IndexError) as e:
        print("AttributeError:", e)
        question_list.append("AttributeError")
        continue

  except genai.errors.ServerError as e: # Change to the correct exception type
    print(f"Server error for image: {ID}, reason: {e}")
    question_list.append("Lỗi máy chủ")
    continue  # Skip to the next image



Nature_000000000290
Nature_000000000291
Nature_000000000292
Nature_000000000293
Nature_000000000294
Nature_000000000295
Nature_000000000296
Nature_000000000297
Nature_000000000298
Nature_000000000299
Nature_000000000300
Nature_000000000301
Nature_000000000302
Nature_000000000303
Nature_000000000304
Nature_000000000305
Nature_000000000306
Nature_000000000307
Nature_000000000308
Nature_000000000309
Nature_000000000310
Server error for image: Nature_000000000310, reason: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The service is currently unavailable.', 'status': 'UNAVAILABLE'}}
Nature_000000000311
Nature_000000000312
Nature_000000000313
Nature_000000000314
Nature_000000000315
Nature_000000000316
Nature_000000000317
Nature_000000000318
Nature_000000000319
Nature_000000000320
Nature_000000000321
Nature_000000000322
Nature_000000000323
Nature_000000000324
Nature_000000000325
Server error for image: Nature_000000000325, reason: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'Th

In [22]:
# imageID_list

In [24]:
# def find_empty_lists(input_list):
#        empty_list_indices = []
#        for index, element in enumerate(input_list):
#            if isinstance(element, list) and len(element) == 0:  # Check if it's a list and empty
#                empty_list_indices.append(index)
#        return empty_list_indices



In [25]:
questions_result_list = []
num_ques_list = []
for result in question_list:
  questions, num_ques = extract_and_count(result)
  questions_result_list.append(questions)
  num_ques_list.append(num_ques)

In [26]:
# questions_result_list

In [ ]:
# prompt: Write caption_list and ImageID_list to CSV file

import csv

with open('path', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['ImageID', 'Question','Num_Ques'])  # Write header row
  for i in range(len(imageID_list)):
    writer.writerow([imageID_list[i], questions_result_list[i], num_ques_list[i]])

## Answer Generation - Gemini

In [ ]:
import pandas as pd

question_df = pd.read_csv('path') #path to question_csv
question_df

,ImageID,Question,Num_Ques
0,Education_000000000001,['Câu hỏi 1: Người phụ nữ trong ảnh mặc trang ...,5
1,Education_000000000002,['Câu hỏi 1: Cô bé trên sân khấu đang làm gì?'...,5
2,Education_000000000003,['Câu hỏi 1: Các học sinh đang làm gì trong lớ...,5
3,Education_000000000004,['Câu hỏi 1: Trong ảnh có những đối tượng nào?...,5
4,Education_000000000005,['Câu hỏi 1: Người phụ nữ trong ảnh đang làm g...,5
...,...,...,...
987,Education_000000000996,['Câu hỏi 1: Các bạn nhỏ trong hình đang thực ...,5
988,Education_000000000997,['Câu hỏi 1: Các bạn nhỏ trong ảnh đang thực h...,5
989,Education_000000000998,['Câu hỏi 1: Các bạn học sinh đang mặc trang p...,5
990,Education_000000000999,['Câu hỏi 1: Các em bé trong ảnh đang mặc tran...,5


In [21]:
question_df_list = question_df['Question'].tolist()

In [22]:
model_name = "gemini-2.0-flash"

ans_system_instructions = """
Bạn được cung cấp một bộ các câu hỏi đi kèm với hình ảnh tương ứng, được định dạng như sau:
Ảnh:
['Câu hỏi 1:Nội dung câu hỏi 1',
'Câu hỏi 2:Nội dung câu hỏi 2',
...,
'Câu hỏi k:Nội dung câu hỏi k'.]


Chỉ phản hồi bằng Tiếng Việt và tuân thủ định dạng sau (Quan trọng! Chú ý phản hồi đúng định dạng):
Ảnh:
Câu trả lời cho câu hỏi 1,
Câu trả lời cho câu hỏi 2,
...,
Câu trả lời cho câu hỏi k
      """

safety_settings = [
    types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="BLOCK_ONLY_HIGH",
    ),
]

In [23]:
prompt_ans = """Bạn là một chuyên gia giàu kinh nghiệm trong lĩnh vực phân tích hình ảnh và gán nhãn dữ liệu.

Nhiệm vụ của bạn là:
- Xem xét kỹ nội dung của từng hình ảnh.
- Phân tích và đưa ra câu trả lời chính xác cho từng câu hỏi dựa trên thông tin trực quan trong ảnh.
- Nếu không chắc chắn hoặc không có đủ thông tin để trả lời, hãy phản hồi là 'Tôi không biết đáp án' cho câu hỏi đó.

Yêu cầu:
- Chỉ đưa ra các câu trả lời chắc chắn và có cơ sở dựa trên hình ảnh.
- Đảm bảo câu trả lời của bạn ngắn gọn, rõ ràng và hoàn toàn phù hợp với nội dung của hình ảnh."""

In [ ]:
imageID_list = []
answer_list = []

start = 0
end = 1000

for index, path in enumerate(image_paths[start:end]):
  ID = convert_name(path)
  imageID_list.append(ID)
  print(ID) #Check
  image = Image.open(path)
  contents = [prompt_ans] + [image] + [question_list[index]]

  try:
    response = client.models.generate_content(
        model = model_name,
        contents = contents,
        config = types.GenerateContentConfig(
            system_instruction=ques_system_instructions,
            temperature=0.4,
            safety_settings=safety_settings,
            )
        )

    try:
    # Check if 'candidates' list is not empty
      if response.candidates:
          # Access the first candidate's content if available
          if response.candidates[0].content.parts:
              generated_text = response.candidates[0].content.parts[0].text
              answer_list.append(generated_text)
              time.sleep(3)
          else:
              print("No generated text found in the candidate.")
              answer_list.append("Không có chú thích")
      else:
          print("No candidates found in the response.")
          answer_list.append("Không nhận được phản hổi")
    except (AttributeError, IndexError) as e:
        print("AttributeError:", e)
        answer_list.append("AttributeError")
        continue

  except genai.errors.ServerError as e: # Change to the correct exception type
    print(f"Server error for image: {ID}, reason: {e}")
    question_list.append("Lỗi máy chủ")
    continue  # Skip to the next image



In [ ]:
answer_list

['Ảnh:\nCâu hỏi 1: Cô giáo trong ảnh đang mặc áo dài màu xanh.\nCâu hỏi 2: Các em học sinh đang cầm cờ, hoa và bó hoa.\nCâu hỏi 3: Quốc kỳ trong ảnh có màu đỏ và vàng.\nCâu hỏi 4: Bối cảnh trong ảnh gợi ý về dịp lễ kỷ niệm ngày Nhà giáo Việt Nam.\nCâu hỏi 5: Các em học sinh có vẻ mặt tươi vui, rạng rỡ.\n      ',
 'Ảnh:\nCâu hỏi 1: Cô bé trên sân khấu đang hát.\nCâu hỏi 2: Các bạn học sinh đang ngồi ở dưới sân khấu.\nCâu hỏi 3: Có hai cây xanh ở phía sau sân khấu.\nCâu hỏi 4: Trên sân khấu có những ký hiệu nốt nhạc.\nCâu hỏi 5: Các bạn học sinh đang ngồi trên những chiếc ghế màu đỏ.',
 'Ảnh:\nCâu hỏi 1: Các học sinh đang làm bài kiểm tra.\nCâu hỏi 2: Các học sinh đang mặc đồng phục màu trắng.\nCâu hỏi 3: Có hai học sinh đang đeo nơ trên đầu.\nCâu hỏi 4: Các học sinh đang ngồi ở bàn học.\nCâu hỏi 5: Học sinh đang cầm bút trên tay.\n      ',
 'Ảnh:\nCâu hỏi 1: Cô bé đang nói về nghề cảnh sát phòng cháy, chữa cháy.\nCâu hỏi 2: Người lính cứu hoả đang phun nước dập lửa.\nCâu hỏi 3: Các bạn 

In [ ]:
answers_result_list = []
num_ans_list = []


for result in answer_list:
  answers, num_ans = extract_and_count(result)
  answers_result_list.append(answers)
  num_ans_list.append(num_ans)

In [ ]:
answers_result_list[0]

['Câu hỏi 1: Cô giáo trong ảnh đang mặc áo dài màu xanh.',
 'Câu hỏi 2: Các em học sinh đang cầm cờ, hoa và bó hoa.',
 'Câu hỏi 3: Quốc kỳ trong ảnh có màu đỏ và vàng.',
 'Câu hỏi 4: Bối cảnh trong ảnh gợi ý về dịp lễ kỷ niệm ngày Nhà giáo Việt Nam.',
 'Câu hỏi 5: Các em học sinh có vẻ mặt tươi vui, rạng rỡ.']

In [ ]:
# empty_list = []
# empty_list = empty_list + find_empty_lists(answers_result_list)
# empty_list = empty_list + find_empty_lists(questions_result_list)
# empty_list = list(set(empty_list))
# empty_list

In [ ]:
# for index in empty_list:
#   questions_result_list.pop(index)
#   answers_result_list.pop(index)
#   imageID_list.pop(index)

In [ ]:
# prompt: Write caption_list and ImageID_list to CSV file

import csv

with open('path', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['ImageID', 'Answer', 'Num_Ans'])  # Write header row
  for i in range(len(imageID_list)):
    writer.writerow([imageID_list[i], answers_result_list[i], num_ans_list[i]])


In [ ]:
# prompt: Write caption_list and ImageID_list to CSV file

import csv

with open('path', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['ImageID', 'Question','Num_Ques', 'Answer', 'Num_Ans'])  # Write header row
  for i in range(len(imageID_list)):
    writer.writerow([imageID_list[i], questions_result_list[i], num_ques_list[i], answers_result_list[i], num_ans_list[i]])


## VLM
